### IMPORT NECESSARY LIBRARIES

In [52]:

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

### LOAD THE DATASET

In [57]:
df = pd.read_csv("Sentiment.csv")

In [58]:
print(df.columns)


Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')


In [59]:
new_df = df[['text','sentiment']]

print(df.sentiment)


0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [60]:
new_df['sentiment'].value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [61]:
new_df = new_df[new_df.sentiment != "Neutral"]


### Print the total number of positive and negative sentiments. 

In [62]:
new_df['sentiment'].value_counts()

Negative    8493
Positive    2236
Name: sentiment, dtype: int64

### Build a sequential LSTM model to predict positive and negative sentiments.

In [65]:
embed_dim = 128
lstm_out = 196
vocabSize=25768
model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1, 128)            3298304   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 1, 128)           0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 196)               254800    
                                                                 
 dense_2 (Dense)             (None, 2)                 394       
                                                                 
Total params: 3,553,498
Trainable params: 3,553,498
Non-trainable params: 0
_________________________________________________________________
None


In [66]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9119, 1) (9119, 2)
(1610, 1) (1610, 2)


In [67]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
285/285 - 13s - loss: 0.0584 - accuracy: 0.9906 - 13s/epoch - 47ms/step
Epoch 2/10
285/285 - 10s - loss: 2.0518e-04 - accuracy: 1.0000 - 10s/epoch - 34ms/step
Epoch 3/10
285/285 - 9s - loss: 7.6656e-05 - accuracy: 1.0000 - 9s/epoch - 32ms/step
Epoch 4/10
285/285 - 9s - loss: 3.9956e-05 - accuracy: 1.0000 - 9s/epoch - 32ms/step
Epoch 5/10
285/285 - 9s - loss: 2.3782e-05 - accuracy: 1.0000 - 9s/epoch - 32ms/step
Epoch 6/10
285/285 - 11s - loss: 1.6167e-05 - accuracy: 1.0000 - 11s/epoch - 37ms/step
Epoch 7/10
285/285 - 9s - loss: 1.1117e-05 - accuracy: 1.0000 - 9s/epoch - 31ms/step
Epoch 8/10
285/285 - 8s - loss: 8.0960e-06 - accuracy: 1.0000 - 8s/epoch - 29ms/step
Epoch 9/10
285/285 - 8s - loss: 6.1454e-06 - accuracy: 1.0000 - 8s/epoch - 29ms/step
Epoch 10/10
285/285 - 8s - loss: 4.7523e-06 - accuracy: 1.0000 - 8s/epoch - 29ms/step


In [68]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

51/51 - 0s - loss: 1.9374e-06 - accuracy: 1.0000 - 447ms/epoch - 9ms/step
score: 0.00
acc: 1.00


In [69]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0

for x in range(len(X_test)):
    
    result = model.predict(X_test[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_test[x]):
        if np.argmax(Y_test[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_test[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 261ms/epoch - 261ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 1

1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14m

1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12m

1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 11ms/epoch - 11ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14m

1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 11ms/epoch - 11ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13m

1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13m

1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 12ms/epoch - 12ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 18m

### Based on the model, check the sentiment for the following two sentences
#### a. 'He is a great leader.'
#### b. 'He is a terrible leader.'

In [73]:
twt = ["He is a great leader"]

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("positive")
elif (np.argmax(sentiment) == 1):
    print("negative")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 0s - 26ms/epoch - 26ms/step
positive


In [72]:
twt = ["He is a terrible leader"]

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 0s - 24ms/epoch - 24ms/step
negative
